# Leabra Units 

This notebook recreates the ["Neuron" tutorial of Chapter 2 of the CCN textbook](https://grey.colorado.edu/CompCogNeuro/index.php/CCNBook/Sims/Neuron/Neuron). It uses the Leabra framework corresponding to [emergent](https://grey.colorado.edu/emergent) 8.0. 

In [1]:
import dotdot
import leabra
import graphs

<IPython.core.display.Javascript object>

In [2]:
receiver = leabra.Unit()

In [3]:
receiver.show_config()

Constants:
   dt_vm: 0.10
   dt_net: 0.70
   g_l: 1.00
   g_bar_e: 0.40
   g_bar_l: 2.80
   g_bar_i: 1.00
   e_rev_e: 1.00
   e_rev_l: 0.15
   e_rev_i: 0.15
   act_thr: 0.25
   act_gain: 600.00
State:
   g_e: 0.00
   I_net: 0.00
   v_m: 0.15
   act: 0.00


To make things as simple as possible, we do not use a sender unit or a network instance. Instead, we manually provide inputs to the receiver unit. The simulation last 200ms. All inputs are 0.0, exepts every input between 10ms and 160ms, which are 1.0.

In [4]:
inputs = 10*[0.0] + 150*[1.0] + 40*[0.0]

for g_e in inputs:
    receiver.add_excitatory(g_e)
    receiver.cycle()

We can monitor the excitatory input the unit receives (`net`), its membrane potential (`v_m`), its total conductance (`I_net`) and its ouput activity (`act`). 

In [5]:
graphs.unit_activity(receiver.logs)

## Manipulating Parameters

You can use sliders to easily modify the `g_e_bar`, `g_l_bar`, `e_rev_e` and `e_rev_l`as in the [CECN1 tutorial](https://grey.colorado.edu/CompCogNeuro/index.php/CECN1_Units#Manipulating_Parameters). Do refer to the tutorial for a discussion about interesting this to try with those parameters.

In [6]:
figdata = graphs.unit_activity_interactive(receiver.logs)

# sliders
g_bar_e_slider = graphs.floatslider(min=0.0, max=1.0, step=0.01, value=0.40)
g_bar_l_slider = graphs.floatslider(min=0.0, max=4.0, step=0.01, value=2.80)
e_rev_e_slider = graphs.floatslider(min=0.0, max=1.0, step=0.01, value=1.00)
e_rev_l_slider = graphs.floatslider(min=0.1, max=0.3, step=0.001, value=0.15)

def regenerate_activity(g_bar_e, g_bar_l, e_rev_e, e_rev_l):
    """Recompute the graph with given paremeters values"""
    receiver = leabra.Unit()
    receiver.spec.g_bar_e = g_bar_e
    receiver.spec.g_bar_l = g_bar_l
    receiver.spec.e_rev_e = e_rev_e
    receiver.spec.e_rev_l = e_rev_l
    
    for g_e in inputs:
        receiver.add_excitatory(g_e)
        receiver.cycle()
        
    graphs.unit_activity_interactive(receiver.logs, figdata=figdata)

In [7]:
graphs.interact(regenerate_activity, g_bar_e=g_bar_e_slider, g_bar_l=g_bar_l_slider, 
                                     e_rev_e=e_rev_e_slider, e_rev_l=e_rev_l_slider)